<h1 style='color:#5E7AFF'>OLMS-Riordan numbers</h1>
<p>This SageMath notebook is the companion of this <a href='http://olms.onl/Riordan/OLMS-Riordan.html'>OLMS PAGE</a>. You can download the notebook from <a href='https://github.com/OpenLibMathSeq/OLMS/tree/master/Notebooks/'>GitHub</a>.</p>


In [ ]:
load('http://olms.onl/Sagemath/OLMS-notebook.sage')


<h2 style='color:#5E7AFF'>Sage class</h2>


In [ ]:
class RiordanClass(OLMS1):
    def generator(self):   # generator
        a, b, n = 1, 0, 1
        yield a
        while True:
            yield b
            n += 1
            a, b = b, (2*b+3*a)*(n-1)//(n+1)

We assume that the reference below is declared throughout the notebook.


In [ ]:
Riordan = RiordanClass()

Note that the class provides functions which are more efficient than the use of list comprehensions.


In [ ]:
print[Riordan(n) for n in range(12)] # slow
print Riordan.sequence(12) # via generator (fast)

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>1</span></h2>
$$A_{n} \ = \ \frac{n-1}{n+1}(2\,A_{n-1}+3\,A_{n-2})$$

In [ ]:
@cached_function
def F1(n):
    if n==0: return 1
    if n==1: return 0
    return (2*F1(n-1)+3*F1(n-2))*(n-1)//(n+1)
print [F1(n) for n in range(12)]

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>2</span></h2>
$$A_{n} \ = \ [x^n] \frac{2}{1+x+\sqrt {(1+x)(1-3\,x)}}$$

In [ ]:
F2 = 2/(1+x+sqrt((1+x)*(1-3*x)))
print taylor(F2, x, 0, 11).list()

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>3</span></h2>
$$A_{n} \, = \, n!\, [x^n]\, e^{x} (\operatorname{I}_0(2\,x) - \operatorname{I}_1(2\,x))$$

In [ ]:
F3 = exp(x)*(bessel_I(0, 2*x)-bessel_I(1, 2*x))
print [factorial(n)*c for n, c in enumerate(taylor(F3, x, 0, 11).list())]

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>4</span></h2>
$$A_{n} \ = \  (-1)^{n} \operatorname{F}\left(\genfrac{}{\vert}{0pt}{}{-n,1/2}{2} 4\right)$$

In [ ]:
F4 = lambda n: (-1)^n*hypergeom([-n,1/2],[2],4)
check(F4, Riordan, 12)

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>5</span></h2>
$$A_{n} \ = \  \frac{(-1)^{n}}{n+1} \operatorname{P}_n^{(1,\,-n-3/2)}\, (-7)$$

In [ ]:
F5 = lambda n: (-1)^n*jacobi_P(n,1,-n-3/2,-7)//(n+1)
check(F5, Riordan, 12)

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>6</span></h2>
$$A_{n} \ = \  \sum \limits_{k=0}^{n} \binom{n}{k} \left(\binom{k}{n-k} - \binom{k}{n-k-1} \right)$$

In [ ]:
F6 = lambda n: sum(binomial(n,k)*(binomial(k,n-k)-binomial(k,n-k-1)) for k in (0..n))
check(F6, Riordan, 12)

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>7</span></h2>
$$A_{n} \ = \  \sum \limits_{k=0}^{n} \binom{n}{k}\binom{k-1}{2k-n}\frac{1}{k+1}$$

In [ ]:
F7 = lambda n: sum(binomial(n,k)*binomial(k-1,2*k-n)//(k+1) for k in (0..n))
check(F7, Riordan, 12)

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>8</span></h2>
$$A_{n+1} \ = \  M_{n} - A_{n}$$

In [ ]:
M = MotzkinClass().generator()
R = RiordanClass().generator()
print [M.next() - R.next() for _ in range(12)]
print Riordan.sequence(13)[1:]

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>9</span></h2>
$$A_{n+2} \ = \  \sum \limits_{k=0}^{n} M_{n,\,k} $$

In [ ]:
@cached_function
def M(n, k):
   if k<0 or n<k: return 0
   if k==0: return 0^n
   return M(n,k-1) + M(n-1,k-1) + M(n-2,k-1)

F9 = lambda n: sum(M(n,k) for k in (0..n))
R = Riordan.generator(); R.next(); R.next();
check(F9, R, 12)

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>10</span></h2>
$$\mathbb{A}_n(1) \ = \  \sum \limits_{k=0}^{n}(-1)^{k}\binom{n+1}{k+1} \binom{k}{\lfloor{k/2}\rfloor}$$

In [ ]:
F10_lhs = partial_sum(Riordan.generator())
F10_rhs = lambda n: sum((-1)^k*binomial(n+1,k+1)*binomial(k,k//2) for k in (0..n))
check(F10_lhs, F10_rhs, 12)

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>11</span></h2>
$$\mathbb{A}_n(A_k) \ = \  A_{n+1} + (-1)^{n} \ = \  \sum \limits_{k=0}^{n} (-1)^{n-k}\,M_k$$

In [ ]:
F11_lhs = self_convolution(Riordan.generator())
F11_rhs = alternating_sign(alternating_sum(MotzkinClass().generator()))
check(F11_lhs, F11_rhs, 12)

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>12</span></h2>
$$\mathbb{B}_n(A_k) \ = \  C_n$$

In [ ]:
F12_lhs = binomial_trans(Riordan.generator())
F12_rhs = CatalanClass().generator()
check(F12_lhs, F12_rhs, 12)

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>13</span></h2>
$$\mathbb{B}_n^{*}(C_k) \ = \  A_{n}$$

In [ ]:
F13 = binomial_invtrans(CatalanClass().generator())
check(F13, Riordan.generator(), 12)

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>14</span></h2>
$$\mathbb{B}_n^{*}(A_k) \ = \  (-1)^n \binom{n}{\lfloor{n/2}\rfloor}$$

In [ ]:
F14_lhs = binomial_invtrans(Riordan.generator())
F14_rhs = lambda n: (-1)^n*binomial(n,n//2)
check(F14_lhs, F14_rhs, 12)

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>15</span></h2>
$$A(z) \ = \  F\left(\genfrac{}{\vert}{0pt}{}{-z,1/2}{2} 4\right)e^{i \pi z}$$

In [ ]:
A = lambda z: exp(I*pi*z)*hypergeom([-z,1/2],[2],4)
print [A(z) for z in (0..12)]

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>16</span></h2>
$$A(z) \ = \  \frac{1}{\pi} { \int_{0}^{4 }\! (x - 1)^{z} \sqrt{\frac{1}{x} - \frac{1}{4}} \, dx}$$

In [ ]:
F16 = lambda z: (1/pi)*integral((x-1)^z*sqrt(1/x-1/4), x,0,4)
print [F16(z) for z in (0..12)]

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>17</span></h2>
$$A(z) \ = \  \frac{2}{\pi} {\int_{0}^{\pi }\! \left( 4\, \cos^{2}(x)-1 \right)^{z} \sin^{2}(x) \, dx}$$

In [ ]:
F17 = lambda z: (2/pi)*integral((4*cos(x)^2-1)^z*sin(x)^2,x,0,pi)
print [F17(z) for z in (0..12)]

<h2 style='color:#5E7AFF;margin-bottom:16px'>Formula <span style='color:orange'>18</span></h2>
$$A(z) \ \sim \ \frac{3^{z+2}}{8\,z \sqrt{3 z \pi}} \left(1 - \frac{21}{16 z} + O(z^{-2})\right)$$

In [ ]:
A18 = lambda z: 3^(z+2)/(8*z*sqrt(3*z*pi))*(1-21/(16*z))
print [ceil(A18(z)) for z in (1..12)]